In [1]:
import os
import json
from PIL import Image
from utils import print_progress_bar
import numpy as np
import copy
import random

In [20]:
def cut(ds, n):
    return ds[:n]


def join_labels(ds):
    # old_labels = ['plain nature', 'detailed nature', 'Agriculture', 'villages', 'city']
    # new_labels = ['nature', 'country', 'city']
    new_ds = []

    for obj in ds:
        label = obj["label"]
        if isinstance(label, str):
            label = [label]

        if 'city' in label:
            new_label = 'city'
        elif ('Agriculture' in label) or ('villages' in label) or ('country' in label):
            new_label = 'country'
        elif ('plain nature' in label) or ('detailed nature' in label) or ('nature' in label):
            new_label = 'nature'
        else:
            continue

        # Update the label in the dataset object
        obj['label'] = new_label

        # Update the file path
        old_path = obj["path"]
        folder, filename = os.path.split(old_path)
        parent_folder = os.path.dirname(folder)
        new_folder = os.path.join(parent_folder, new_label)
        new_path = os.path.join(new_folder, filename)

        # Move the file to the new path
        # shutil.move(old_path, new_path)

        # Update the path in the dataset object
        obj["path"] = new_path

        new_ds.append(obj)

    return new_ds


def shuffle_dataset(ds):
    """Shuffles the given dataset list randomly."""
    random.shuffle(ds)
    return ds


def augment_data(ds):
    new_ds = []
    for obj in ds:
        cop_obj = copy.deepcopy(obj)
        for i in range(4):
            for j in range(2):
                for method in cop_obj['entropy_results']:
                    if method['method'] == 'dwt':
                        continue
                    else:
                        arr = np.array(method['result'], dtype=object)
                        # Determine the axes based on the shape of arr
                        if arr.ndim == 3:
                            axes_to_rotate = (1, 2)
                        elif arr.ndim == 2:
                            axes_to_rotate = (0, 1)
                        else:
                            continue  # Skip if the array has unexpected dimensions

                        # Rotate the array by 90 degrees 'i' times
                        arr = np.rot90(arr, k=i, axes=axes_to_rotate)
                        # Flip the array if j is 1
                        if j:
                            arr = np.flip(arr, axis=0)
                        method['result'] = arr.tolist()
                new_ds.append(cop_obj)
    return new_ds


def count_label(ds):
    labels = {'total': 0,
              'nature': 0,
              'country': 0,
              'city': 0,
              'villages': 0,
              'detailed nature': 0,
              'plain nature': 0,
              'Agriculture': 0,
              'city-old': 0}
    for obj in ds:
        labels[obj["label"]] += 1
        labels['total'] += 1

    for key, val in labels.items():
        print(key, val)



def augment_images_in_folder(root_folder):
    total_images = sum([len(files) for r, d, files in os.walk(root_folder)])
    processed_images = 0

    for subdir, _, files in os.walk(root_folder):
        for file in files:
            if not file.endswith('.png'):
                continue

            original_path = os.path.join(subdir, file)
            folder, filename = os.path.split(original_path)

            # Load the original image
            try:
                img = Image.open(original_path).convert('RGB')
            except FileNotFoundError:
                print(f"\nFile not found: {original_path}")
                continue

            # Generate augmented images
            for angle in [0, 90, 180, 270]:
                for flip in [False, True]:
                    # Generate new path
                    new_filename = f"{os.path.splitext(filename)[0]}_rot{angle}_flip{int(flip)}.png"
                    new_path = os.path.join(folder, new_filename)

                    # Skip if the image already exists
                    if os.path.exists(new_path):
                        continue

                    # Rotate and flip
                    new_img = img.rotate(angle)
                    if flip:
                        new_img = new_img.transpose(Image.FLIP_LEFT_RIGHT)

                    # Save the new image
                    new_img.save(new_path)

            processed_images += 1
            print(f"Images processed: {processed_images}/{total_images}")

In [16]:
path = "../datasets/all_data/entropy_results.json"
with open(path, 'r') as f:
    dataset = json.load(f)
print("JSON loaded.")

JSON loaded.


In [21]:
count_label(dataset)

total 11812
nature 0
country 800
city 3517
villages 1094
detailed nature 1637
plain nature 1876
Agriculture 1672
city-old 1216


In [26]:
new_dataset = augment_data(dataset)
# new_dataset = join_labels(dataset)
# new_dataset = cut(dataset, 256)
print("Dataset modification done.")

Dataset modification done.


In [27]:
count_label(new_dataset)

total 84768
nature 28104
country 28528
city 28136
villages 0
detailed nature 0
plain nature 0
Agriculture 0
city-old 0


In [28]:
path = "../datasets/all_data/entropy_results_augmented.json"
with open(path, 'w') as f:
    json.dump(new_dataset, f, indent=4)
print("New JSON saved.")

New JSON saved.


In [25]:
dataset = new_dataset